In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
from scipy.special import expit
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def backdoor_adjustment(Y, A, Z, data):
    """
    Compute the average causal effect E[Y(A=1)] - E[Y(A=0)] via backdoor adjustment
    
    Inputs
    ------
    Y: string corresponding variable name of the outcome
    A: string corresponding variable name
    Z: list of variable names to adjust for
    data: pandas dataframe
    
    Return
    ------
    ACE: float corresponding to the causal effect
    """
    
    formula = Y + "~" + A
    if len(Z) > 0:
        formula += " + " + "+".join(Z)
    
    model = sm.GLM.from_formula(formula=formula, data=data, family=sm.families.Gaussian()).fit()
    data_A0 = data.copy()
    data_A1 = data.copy()
    data_A0[A] = 0
    data_A1[A] = 1
    return(np.mean(model.predict(data_A1)) - np.mean(model.predict(data_A0)))

def get_numpy_matrix(data, variables):
    """
    Takes a pandas dataframe and a list of variable names, and returns
    just the raw matrix for those specific variables
    """
    
    
    matrix = data[variables].to_numpy()

    # if there's only one variable, ensure we return a matrix with one column
    # rather than just a column vector
    if len(variables) == 1:
        return matrix.reshape(len(data),)
    return matrix
    
    
def backdoor_ML(Y, A, Z, data):
    """
    Compute the average causal effect E[Y(A=1)] - E[Y(A=0)] via backdoor adjustment with random forests.
    
    Inputs
    ------
    Y: string corresponding variable name of the outcome
    A: string corresponding variable name
    Z: list of variable names to adjust for
    data: pandas dataframe
    
    Return
    ------
    ACE: float corresponding to the causal effect
    """
    
    # some starter code to get the matrix for doing the regression
    # in the right shape
    outcome = get_numpy_matrix(data, [Y])
    predictors = get_numpy_matrix(data, [A] + Z)

    # use a RandomForestRegressor with bootstrap=False

    # get predictions using datasets where A=1 and A=0

    return 1

def aipw_ML(Y, A, Z, data):
    """
    Compute the average causal effect E[Y(A=1)] - E[Y(A=0)] via augmented IPW with random forests.
    
    Inputs
    ------
    Y: string corresponding variable name of the outcome
    A: string corresponding variable name
    Z: list of variable names to adjust for
    data: pandas dataframe
    
    Return
    ------
    ACE: float corresponding to the causal effect
    """
    
    # some starter code to get the matrix for doing the regressions
    # in the right shape
    treatment = get_numpy_matrix(data, [A])
    predictors_a = get_numpy_matrix(data, Z)
    
    # use a RandomForestClassifier with bootstrap=False for fitting a model for p(A|Z)
    
    # use p(A|Z) model to predict propensity scores and get IPW weights
    
    
    # fit a RandomForestRegressor model for E[Y|A, Z, W] where W are the IPW weights
    
    # get predictions using datasets where A=1 and A=0

    return 1

def backdoor_mean(Y, A, Z, value, data):
    """
    Compute the counterfactual mean E[Y(a)] for a given value of a via backdoor adjustment
    
    Inputs
    ------
    Y: string corresponding variable name of the outcome
    A: string corresponding variable name
    Z: list of variable names to adjust for
    value: float corresponding value to set A to
    data: pandas dataframe
    
    Return
    ------
    ACE: float corresponding to the causal effect
    """
    
    formula = Y + "~" + A
    if len(Z) > 0:
        formula += " + " + "+".join(Z)
    
    model = sm.GLM.from_formula(formula=formula, data=data, family=sm.families.Gaussian()).fit()
    data_a = data.copy()
    data_a[A] = value
    return np.mean(model.predict(data_a))
    
def compute_confidence_intervals(Y, A, Z, data, method_name, num_bootstraps=100, alpha=0.05, value=None):
    """
    Compute confidence intervals for backdoor adjustment via bootstrap
    
    Returns tuple (q_low, q_up) for the lower and upper quantiles of the confidence interval.
    """
    
    Ql = alpha/2
    Qu = 1 - alpha/2
    estimates = []
    
    for i in range(num_bootstraps):
        
        # resample the data with replacement
        data_sampled = data.sample(len(data), replace=True)
        data_sampled.reset_index(drop=True, inplace=True)
        
        # add estimate from resampled data
        if method_name == "backdoor":
            estimates.append(backdoor_adjustment(Y, A, Z, data_sampled))
        
        elif method_name == "backdoor_ML":
            estimates.append(backdoor_ML(Y, A, Z, data_sampled))
            
        elif method_name == "aipw_ML":
            estimates.append(aipw_ML(Y, A, Z, data_sampled))
            
        elif method_name == "backdoor_mean":
            estimates.append(backdoor_mean(Y, A, Z, value, data_sampled))

        else:
            print("Invalid method")
            estimates.append(1)

    # calculate the quantiles
    quantiles = np.quantile(estimates, q=[Ql, Qu])
    q_low = quantiles[0]
    q_up = quantiles[1]
    
    return q_low, q_up

# Exercise

The code block below contains code to generate data from a simple model where $A \leftarrow Z \rightarrow Y$ and $A\rightarrow Y.$ That is, $Z=\{Z_1, Z_2, Z_3\}$ is a valid adjustment set for the effect of $A$ on $Y.$ However, the relations between these variables are non-linear and so backdoor adjustment/IPW with linear models is expected to produce biased estimates. The true average causal effect is 2. We will see if we are able to recover this causal effect using machine learning.

1. Implement the `backdoor_ML` function that uses a `RandomForestRegressor` to fit $\mathbb{E}[Y\mid A, Z].$ How do the estimates from this method compare to estimates obtained by using linear models for $n=200, 2000, 20000$ samples? What does this say about plug-in bias?


2. Implement the `aipw_ML` function that uses a `RandomForestClassifier` for the propensity score model $p(A\mid Z)$ and a `RandomForestRegressor` for the outcome regression $\mathbb{E}[Y \mid A, Z].$ Evaluate the causal effect using the *aipw\_ML* method at the same sample sizes as before. Does it help protect against plug-in bias?

In [ ]:
np.random.seed(0)

# data generating process
size = 200
Z1 = np.random.normal(0, 1, size)
Z2 = np.random.binomial(1, 0.5, size)
Z3 = np.random.uniform(0, 1, size)
A = np.random.binomial(1, expit(Z1*Z3 + Z1*Z2 + Z1**2), size)
Y = 1.5 + 2.5*Z1*Z2*Z3 + Z1**2 + Z2**2 + 2*A + np.random.normal(0, 1, size)
data = pd.DataFrame({"Y": Y, "A": A, "Z1": Z1, "Z2": Z2, "Z3": Z3})

# point estimate and confidence intervals using linear models in backdoor adjustment
point_estimate = backdoor_adjustment("Y", "A", ["Z1", "Z2", "Z3"], data)
cis = compute_confidence_intervals("Y", "A", ["Z1", "Z2", "Z3"], data, method_name="backdoor")
print(point_estimate, cis)
np.mean(A)

# point estimate using machine learning in backdoor adjustment
point_estimate = backdoor_ML("Y", "A", ["Z1", "Z2", "Z3"], data)
cis = compute_confidence_intervals("Y", "A", ["Z1", "Z2", "Z3"], data, method_name="backdoor_ML")
print(point_estimate, cis)

# point estimate using machine learning for reweighted backdoor adjustment
point_estimate = aipw_ML("Y", "A", ["Z1", "Z2", "Z3"], data)
cis = compute_confidence_intervals("Y", "A", ["Z1", "Z2", "Z3"], data, method_name="aipw_ML")
print(point_estimate, cis)

## Dealing with continuous treatment variables

In [ ]:
np.random.seed(0)

# data generating process
size = 200
Z = np.random.uniform(0, 1, size)
A = 1.2*Z + np.random.uniform(0, 2, size)
Y = 1.5 + 2.5*Z + 2*A + A**4 + np.random.normal(0, 1, size)
data = pd.DataFrame({"Y": Y, "A": A, "Z": Z})

point_estimates = []
values_a = list(np.arange(0, 2, 0.2))
for value in values_a:
    point_estimates.append(backdoor_mean("Y", "A", ["Z", "np.power(A, 4)"], value, data))

ci_lower = []
ci_upper = []
for value in values_a:
    ci = compute_confidence_intervals("Y", "A", ["Z", "np.power(A, 4)"],
                                      data, method_name="backdoor_mean", value=value)
    ci_lower.append(ci[0])
    ci_upper.append(ci[1])
    


In [ ]:
from matplotlib import pyplot as plt

point_estimates = np.array(point_estimates)
ci_lower, ci_upper = np.array(ci_lower), np.array(ci_upper)
fig, ax = plt.subplots()
ax.plot(values_a, point_estimates)
ax.fill_between(values_a, ci_lower, ci_upper, color='b', alpha=.1)